In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/shl-intern-hiring-assessment/Dataset/sample_submission.csv
/kaggle/input/shl-intern-hiring-assessment/Dataset/train.csv
/kaggle/input/shl-intern-hiring-assessment/Dataset/test.csv
/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test/audio_885.wav
/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test/audio_1142.wav
/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test/audio_1006.wav
/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test/audio_817.wav
/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test/audio_765.wav
/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test/audio_508.wav
/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test/audio_257.wav
/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test/audio_330.wav
/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test/audio_72.wav
/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test/audio_328.wav
/kaggle/input/shl-intern-hiring-ass

## Installing required libraries

In [3]:
!pip install torch pandas transformers sentencepiece librosa datasets nlpaug scikit-learn numpy openai-whisper

## All Imports

In [4]:
import re
import os
import torch
import whisper
import librosa
import regex as re
import soundfile as sf
from datasets import Dataset
from tqdm.notebook import tqdm
import nlpaug.augmenter.word as naw
from typing import Union, List, Optional
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from transformers import ( AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer )

2025-05-03 22:17:36.717059: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746310656.742065     139 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746310656.749508     139 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Cleaning Training Audio

In [10]:
# Paths
TRAIN_AUDIO_DIR = '/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/train/'
TRAIN_CSV_PATH = '/kaggle/input/shl-intern-hiring-assessment/Dataset/train.csv'
TEST_AUDIO_DIR = '/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test/'
TEST_CSV_PATH = '/kaggle/input/shl-intern-hiring-assessment/Dataset/test.csv'
TRAIN_CLEAN_AUDIO = '/kaggle/working/cleaned_audio_train'
TEST_CLEAN_AUDIO = '/kaggle/working/cleaned_audio_test'

os.makedirs(TRAIN_CLEAN_AUDIO, exist_ok=True)

# Load CSV and rename columns
train_df = pd.read_csv(TRAIN_CSV_PATH)

# Preprocessing function
def preprocess_audio(file_path, save_path, sr=16000):
    y, orig_sr = librosa.load(file_path, sr=None)
    if orig_sr != sr:
        y = librosa.resample(y, orig_sr, sr)
    y = y / max(abs(y))  # Normalise volume
    y, _ = librosa.effects.trim(y, top_db=25)  # Trim silence
    sf.write(save_path, y, sr)

# # Preprocess each training audio
# for filename in tqdm(train_df['filename']):
#     in_path = os.path.join(TRAIN_AUDIO_DIR, filename)
#     out_path = os.path.join(TRAIN_CLEAN_AUDIO, filename)
#     preprocess_audio(in_path, out_path)

In [13]:
test_df = pd.read_csv(TEST_CSV_PATH)
os.makedirs(TEST_CLEAN_AUDIO, exist_ok=True)

# Preprocess each testing audio
# for filename in tqdm(test_df['filename']):
#     in_path = os.path.join(TEST_AUDIO_DIR, filename)
#     out_path = os.path.join(TEST_CLEAN_AUDIO, filename)
#     preprocess_audio(in_path, out_path)

## Extracting Clean Transcripts

In [14]:
class TranscriptCleaner:    
    DEFAULT_FILLERS = [
        'uh', 'um', 'erm', 'you know', 'like', 'i mean', 
        'hmm', 'ah', 'uhh', 'huh', 'well', 'so', 'right',
        'actually', 'basically', 'literally', 'okay'
    ]
    
    PUNCTUATION_MAP = {
        '“': '"', '”': '"', "‘": "'", "’": "'", '–': '-',
        '…': '...', '``': '"', "''": '"'
    }
    
    def __init__(
        self,
        fillers: Optional[List[str]] = None,
        remove_fillers: bool = True,
        normalize_case: bool = True,
        clean_punctuation: bool = True,
        remove_extra_spaces: bool = True,
        remove_special_chars: bool = False,
        custom_patterns: Optional[List[str]] = None
    ):
        self.fillers = fillers or self.DEFAULT_FILLERS
        self.remove_fillers = remove_fillers
        self.normalize_case = normalize_case
        self.clean_punctuation = clean_punctuation
        self.remove_extra_spaces = remove_extra_spaces
        self.remove_special_chars = remove_special_chars
        self.custom_patterns = custom_patterns or []
        
        # Pre-compile regex patterns for better performance
        self._compiled_patterns = {
            'fillers': re.compile(r'\b(?:{})\b'.format('|'.join(map(re.escape, self.fillers)))),
            'punctuation': re.compile('|'.join(map(re.escape, self.PUNCTUATION_MAP.keys()))),
            'extra_spaces': re.compile(r'\s+'),
            'special_chars': re.compile(r'[^\w\s.,?!\'"-]'),
            'leading_trailing_punct': re.compile(r'^[\s\W]+|[\s\W]+$'),
            'custom': [re.compile(pattern) for pattern in self.custom_patterns]
        }
    
    def clean(self, text: Union[str, pd.Series]) -> Union[str, pd.Series]:
        if isinstance(text, pd.Series):
            return text.apply(self._clean_text)
        return self._clean_text(text)
    
    def _clean_text(self, text: str) -> str:
        if not isinstance(text, str) or not text.strip():
            return text
            
        # Apply cleaning steps in optimal order
        if self.normalize_case:
            text = text.lower()
            
        if self.remove_fillers:
            text = self._compiled_patterns['fillers'].sub('', text)
            
        if self.clean_punctuation:
            text = self._compiled_patterns['punctuation'].sub(
                lambda x: self.PUNCTUATION_MAP[x.group()], text
            )
            # Fix spacing around punctuation
            text = re.sub(r'\s([?.!,](?:\s|$))', r'\1', text)
            text = re.sub(r'([\'"])\s', r'\1', text)
            
        for pattern in self._compiled_patterns['custom']:
            text = pattern.sub('', text)
            
        if self.remove_special_chars:
            text = self._compiled_patterns['special_chars'].sub('', text)
            
        if self.remove_extra_spaces:
            text = self._compiled_patterns['extra_spaces'].sub(' ', text)
            text = self._compiled_patterns['leading_trailing_punct'].sub('', text)
            text = text.strip()
            
        return text

cleaner = TranscriptCleaner(
    fillers=TranscriptCleaner.DEFAULT_FILLERS + ['sort of', 'kind of'],
    custom_patterns=[r'\[.*?\]', r'\(.*?\)'],  # Remove text in brackets
    remove_special_chars=True
)


# Sample dirty transcript [For testing]
# dirty_text = "Uh, so I was like... you know, [clears throat] sort of thinking (about stuff)."
# clean_text = cleaner.clean(dirty_text)
# print(f"Before: {dirty_text}")
# print(f"After: {clean_text}")

# features_df['clean_text'] = features_df['text'].astype(str).apply(cleaner.clean)
# print(features_df[['text', 'clean_text']])
# df = features_df
# df.to_csv('text_clean.csv', index=False)

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

if torch.cuda.is_available():
    torch.cuda.empty_cache()

whisper_model = whisper.load_model("turbo", device=device)
print("whisper turbo: ", whisper_model)

Using device: cuda


/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


whisper turbo:  Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0-31): 32 x ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=1280, out_features=1280, bias=True)
          (key): Linear(in_features=1280, out_features=1280, bias=False)
          (value): Linear(in_features=1280, out_features=1280, bias=True)
          (out): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (attn_ln): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=1280, out_features=5120, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=5120, out_features=1280, bias=True)
        )
        (mlp_ln): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_p

In [18]:
# ctal -> clean_text_and_label

train_ctal=[]
for i, row in tqdm(train_df.iterrows(), total=len(train_df)):
    text = whisper_model.transcribe(os.path.join(TRAIN_CLEAN_AUDIO, row['filename']))['text']
    cleaned_text = cleaner.clean(text)
    train_ctal.append({'text' : cleaned_text, 'label' : row['label']})

test_ctal=[]
for i, row in tqdm(test_df.iterrows(), total=len(test_df)):
    text = whisper_model.transcribe(os.path.join(TEST_CLEAN_AUDIO, row['filename']))
    cleaned_text = cleaner.clean(text)
    test_ctal.append({'filename': row['filename'], 'text': cleaned_text})

train_clean_df = pd.DataFrame(train_ctal)
test_clean_df = pd.DataFrame(test_ctal)

# Saving
train_clean_df.to_csv('train_clean.csv', index=False)
test_clean_df.to_csv('test_clean.csv', index=False)

  0%|          | 0/444 [00:00<?, ?it/s]

  0%|          | 0/204 [00:00<?, ?it/s]

In [19]:
def load_data(file_path):
    data = pd.read_csv(file_path)
    texts = data["text"].astype(str).tolist()
    labels = data["label"].tolist()
    return texts, labels

texts, labels = load_data("train_clean.csv")

In [20]:
label_bins = pd.cut(labels, bins=9)  # 9 bins for 1.0-5.0 in 0.5 steps
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, stratify=label_bins, random_state=42
)

# Data augmentation 
def augment_data(texts, labels, augment_factor=0.3):
    aug = naw.ContextualWordEmbsAug(
        model_path='distilbert-base-uncased', 
        action="substitute",
        device=device
    )
    augmented_texts = []
    augmented_labels = []
    
    for text, label in zip(texts, labels):
        augmented_texts.append(str(text))
        augmented_labels.append(label)
        if np.random.rand() < augment_factor:
            try:
                augmented_text = aug.augment(text)
                if isinstance(augmented_text, list):
                    augmented_text = augmented_text[0]
                augmented_texts.append(str(augmented_text))
                augmented_labels.append(label)
            except Exception as e:
                print(f"Augmentation failed for: {text[:50]}... Error: {str(e)}")
                continue
                
    return augmented_texts, augmented_labels

train_texts, train_labels = augment_data(train_texts, train_labels)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [21]:
# Tokenization
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(texts):
    # Ensure input is list of strings
    if isinstance(texts, str):
        texts = [texts]
    return tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors="pt")

# Converting to list of strings if needed
train_texts = [str(text) for text in train_texts]
val_texts = [str(text) for text in val_texts]

# Tokenize in batches
def batch_tokenize(texts, batch_size=32):
    encodings = {'input_ids': [], 'attention_mask': []}
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        batch_enc = tokenizer(batch, truncation=True, padding=True, max_length=128)
        encodings['input_ids'].extend(batch_enc['input_ids'])
        encodings['attention_mask'].extend(batch_enc['attention_mask'])
    return encodings

train_encodings = batch_tokenize(train_texts)
val_encodings = batch_tokenize(val_texts)

In [23]:
# Creating PyTorch Dataset
class GrammarDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float32)
        }
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = GrammarDataset(train_encodings, train_labels)
val_dataset = GrammarDataset(val_encodings, val_labels)


In [24]:
# Loading model for regression
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=1,
    problem_type="regression"
)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=15,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    disable_tqdm=False,
    dataloader_pin_memory=False,
    learning_rate=3e-5,
    weight_decay=0.01,
    eval_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="mae",
    logging_dir="./logs",
    logging_steps=10,
    warmup_steps=50,
    report_to="none",
    fp16=torch.cuda.is_available(),
)

# Computing important Metrics
def compute_metrics(p):
    preds = p.predictions.squeeze()
    return {
        "mae": mean_absolute_error(p.label_ids, preds),
        "rmse": np.sqrt(mean_squared_error(p.label_ids, preds)),
        "accuracy_0.5": np.mean(np.abs(p.label_ids - preds) <= 0.5),
        "pearson" : np.corrcoef(p.label_ids, preds)[0, 1]
    }

# Training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

# Saving model
model.save_pretrained("./grammar_scorer_regression")
tokenizer.save_pretrained("./grammar_scorer_regression")



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss,Mae,Rmse,Accuracy 0.5,Pearson
10,15.530100,15.182824,3.757206,3.893882,0.000000,-0.227573
20,14.060300,11.914783,3.290819,3.449121,0.000000,-0.213737
30,9.468000,6.284563,2.280916,2.504843,0.101124,-0.198719
40,4.482000,2.159256,1.298406,1.469586,0.168539,0.031042
50,1.497400,1.015279,0.864565,1.011909,0.235955,0.156236
60,0.973600,0.927287,0.790362,0.969073,0.348315,0.405670
70,1.061200,1.012645,0.761306,1.013193,0.449438,0.390455
80,0.790900,0.778382,0.673139,0.889852,0.528090,0.503912
90,0.567300,0.839956,0.710778,0.924793,0.516854,0.537961
100,0.387800,0.790511,0.670572,0.897001,0.561798,0.567912


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

('./grammar_scorer_regression/tokenizer_config.json',
 './grammar_scorer_regression/special_tokens_map.json',
 './grammar_scorer_regression/vocab.txt',
 './grammar_scorer_regression/added_tokens.json',
 './grammar_scorer_regression/tokenizer.json')

In [26]:
def predict_grammar(text):
    inputs = tokenizer(str(text), return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {key:val.to(device) for key, val in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    pred_score = outputs.logits.item()
    rounded_score = round(pred_score * 2) / 2
    final_score = max(1.0, min(5.0, rounded_score))
    return final_score

def load_data(file_path):
    data = pd.read_csv(file_path)
    texts = data["text"].astype(str).tolist()
    filenames = data["filename"].tolist()
    return texts, filenames

texts, filenames = load_data("test_clean.csv")

submission_csv = []
for text, filename in tqdm(zip(texts, filenames)):
    label = predict_grammar(text)
    submission_csv.append({'filename' : filename, 'label':label})

submission_csv_df = pd.DataFrame(submission_csv)
submission_csv_df.to_csv('aayush_submission.csv', index=False)

0it [00:00, ?it/s]